Variational AutoEncoder (VAE) with CelebA
=========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Init python stuff

In [1]:
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.24

FIDLE 2020 - DataGenerator
Version              : 0.4


## Step 2 - Prepare data
### 2.1 - Dataset localisation

In [2]:
dataset_dir  = '/bettik/PROJECTS/pr-fidle/datasets/celeba'

train_dir    = f'{dataset_dir}/clusters.train'
test_dir     = f'{dataset_dir}/clusters.test'

### 2.2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [15]:
tag = 'CelebA.001'

input_shape = (128, 128, 3)
z_dim       = 200
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.001
Config saved in     : ./run/CelebA.001/models/vae_config.json


## Step 4 - Compile it

In [16]:
learning_rate = 0.005
r_loss_factor = 10000

vae.compile(learning_rate, r_loss_factor)

Compiled.
Optimizer is Adam with learning_rate=0.005


## Step 5 - Train

In [17]:
epochs            = 6
image_periodicity = 8      # for each epoch
chkpt_periodicity = 1      # for each epoch
initial_epoch     = 0
dataset_size      = 1

In [18]:
vae.train(#x_train     = x_train,
          data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          image_periodicity = image_periodicity,
          chkpt_periodicity = chkpt_periodicity,
          initial_epoch     = initial_epoch
         )

Epoch 1/6
6250/6250 [==============================] - 206s 33ms/step - loss: 338.2761 - vae_r_loss: 298.7408 - vae_kl_loss: 39.5353 - val_loss: 309.0752 - val_vae_r_loss: 276.8837 - val_vae_kl_loss: 31.9974
Epoch 2/6
6250/6250 [==============================] - 209s 33ms/step - loss: 296.6832 - vae_r_loss: 256.9755 - vae_kl_loss: 39.7077 - val_loss: 289.8210 - val_vae_r_loss: 255.9433 - val_vae_kl_loss: 33.6266
Epoch 3/6
6250/6250 [==============================] - 200s 32ms/step - loss: 293.1235 - vae_r_loss: 253.5661 - vae_kl_loss: 39.5573 - val_loss: 302.0345 - val_vae_r_loss: 268.0425 - val_vae_kl_loss: 33.7719
Epoch 4/6
6250/6250 [==============================] - 205s 33ms/step - loss: 292.2448 - vae_r_loss: 252.9195 - vae_kl_loss: 39.3251 - val_loss: 273.3243 - val_vae_r_loss: 239.4294 - val_vae_kl_loss: 33.6550
Epoch 5/6
6250/6250 [==============================] - 216s 35ms/step - loss: 292.4938 - vae_r_loss: 253.2241 - vae_kl_loss: 39.2703 - val_loss: 277.4104 - val_vae_r_lo